<a href="https://colab.research.google.com/github/BharathraajNagarajan/Distributed_Systems_Workflow_Optimizer/blob/main/Spark_Streaming_Checkpoint_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install Java
!apt-get install -y openjdk-11-jdk

# Download Spark 3.5.1 (with Hadoop 3)
!wget -q --show-progress https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz

# Extract Spark
!tar -xzf spark-3.5.1-bin-hadoop3.tgz

# Install PySpark & findspark (matching version)
!pip install -q pyspark==3.5.1 findspark

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

# Initialize findspark
import findspark
findspark.init()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.28+6-1ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
spark-3.5.1-bin-had 100%[===================>] 381.90M  25.8MB/s    in 16s     


In [4]:
!tar -xzf spark-3.5.1-bin-hadoop3.tgz


In [5]:
!pip install -q pyspark==3.5.1 findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

import findspark
findspark.init()

In [7]:
!mkdir -p /content/Distributed_Systems_Workflow_Optimizer/src
!mkdir -p /content/Distributed_Systems_Workflow_Optimizer/data
!mkdir -p /content/Distributed_Systems_Workflow_Optimizer/out


In [11]:
!mkdir -p /content/Distributed_Systems_Workflow_Optimizer/scripts

In [ ]:
from google.colab import files

# Upload Python scripts
uploaded = files.upload()
# Save them manually after selecting (e.g., to src folder)
!mv dijkstra_spark.py /content/Distributed_Systems_Workflow_Optimizer/src/
!mv streaming_checkpoint_demo.py /content/Distributed_Systems_Workflow_Optimizer/src/

# Upload CSV
uploaded = files.upload()
!mv graph_edges.csv /content/Distributed_Systems_Workflow_Optimizer/data/

In [12]:
csv_content = """src,dst,w
A,B,1
A,C,4
B,C,2
B,D,5
C,D,1
D,E,1
E,F,3
F,G,2
F,H,1
G,H,1
"""

with open("/content/Distributed_Systems_Workflow_Optimizer/data/graph_edges.csv", "w") as f:
    f.write(csv_content)


In [14]:
!spark-submit /content/Distributed_Systems_Workflow_Optimizer/src/dijkstra_spark.py \
  --edges /content/Distributed_Systems_Workflow_Optimizer/data/graph_edges.csv \
  --source A \
  --output /content/Distributed_Systems_Workflow_Optimizer/out/distances \
  --maxIters 50 \
  --partitions 4


25/09/21 08:38:25 INFO SparkContext: Running Spark version 3.5.1
25/09/21 08:38:25 INFO SparkContext: OS info Linux, 6.1.123+, amd64
25/09/21 08:38:25 INFO SparkContext: Java version 11.0.28
25/09/21 08:38:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/21 08:38:26 INFO ResourceUtils: ==============================================================
25/09/21 08:38:26 INFO ResourceUtils: No custom resources configured for spark.driver.
25/09/21 08:38:26 INFO ResourceUtils: ==============================================================
25/09/21 08:38:26 INFO SparkContext: Submitted application: DijkstraSpark
25/09/21 08:38:26 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpu

In [15]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.parquet("/content/Distributed_Systems_Workflow_Optimizer/out/distances")
df.show()


+---+----+
| id|dist|
+---+----+
|  F| 8.0|
|  G|10.0|
|  H| 9.0|
|  A| 0.0|
|  D| 4.0|
|  B| 1.0|
|  C| 3.0|
|  E| 5.0|
+---+----+



In [88]:
!mkdir -p /content/Distributed_Systems_Workflow_Optimizer/tmp/stream_in
!mkdir -p /content/Distributed_Systems_Workflow_Optimizer/tmp/chk
!mkdir -p /content/Distributed_Systems_Workflow_Optimizer/out/stream

In [89]:
!rm -rf /content/Distributed_Systems_Workflow_Optimizer/tmp/stream_in/*
!rm -rf /content/Distributed_Systems_Workflow_Optimizer/tmp/chk/*
!rm -rf /content/Distributed_Systems_Workflow_Optimizer/out/*

In [ ]:
!echo "hello spark streaming test" > /content/Distributed_Systems_Workflow_Optimizer/tmp/stream_in/batch1.txt
!echo "spark streaming checkpoint recovery" > /content/Distributed_Systems_Workflow_Optimizer/tmp/stream_in/batch2.txt

In [ ]:
!python /content/Distributed_Systems_Workflow_Optimizer/src/streaming_checkpoint_demo.py \
  --inputDir /content/Distributed_Systems_Workflow_Optimizer/tmp/stream_in \
  --checkpointDir /content/Distributed_Systems_Workflow_Optimizer/tmp/chk \
  --triggerSec 5
